In [1]:
!pip install --upgrade --user --quiet google-cloud-aiplatform
!pip install -qU langchain-google-genai


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import getpass
import os
from dotenv import load_dotenv
import base64
from langchain_core.prompts import HumanMessagePromptTemplate, ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage
from IPython.display import Markdown, display
from langchain_google_genai import ChatGoogleGenerativeAI
from vertexai.preview.generative_models import grounding as preview_grounding
from vertexai.generative_models import (
    GenerationResponse,
    GenerativeModel,
    Tool,
    grounding,
)


In [10]:
load_dotenv()
#os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r"C:\Users\DELL\Downloads\13-Google-GenAI-Hack-24\GenAI-Google--Hack-24\vision-forge-414908-d792f2fc2ff6.json"


In [4]:
PROJECT_ID = "vision-forge-414908"  # @param {type:"string"}
REGION = "us-central1"  # @param {type: "string"}
import vertexai
vertexai.init(project=PROJECT_ID, location=REGION)

In [5]:
import vertexai
vertexai.init(project=PROJECT_ID, location=REGION)

In [8]:
model_ground = GenerativeModel("gemini-1.5-flash-001")

In [14]:
product = "parle g biscuit" 
PROMPT = f"give me the description, ingredietns and flavor profile or taste of this {product}"
tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())
response = model_ground.generate_content(PROMPT, tools=[tool])
#print(response.candidates[0].content.parts[0].text)
food_info = response.candidates[0].content.parts[0].text

In [15]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash-001",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    
)

In [16]:
restrictions = "vegan"

In [51]:
chat_prompt = ChatPromptTemplate.from_messages(
    messages=[
        SystemMessage(
            content='''You are a good great chef and a great cook that can give qyuick and esy recioes followinf sepecific diets, imgredietns, taste of food as well as flvor profiles. 
            You will be given this info and you have to give 2 dishes that mathc the input and give simple recipes that can be made using that product or something similar to that product. If some food dietary
            restrictions are there you are supposed to give recies that can have alternatives from tehe original oproduct but can give similar taste.
            '''),
        HumanMessagePromptTemplate.from_template(
            template= 
                '''The info is the following
                the product name of the major foods in the recipe: {product}. and the major info rgaring the product such as decription, raste and ingredients {food_info}
                Limit the recipes to these dietary restrictions: {restrictions}
                Give only these following info regarding 2 recipes following the above in the following format. The answers for each of the following should be in (** **):
                (#Recipe Name#): short, fun catchy name for recipe
                (#Description#): description of the recipe in not more than 1 line. This should include key worlds like keto, vegan, jain, choletrol less, sugar free, high protein etc.
                no need of any othe info except these 2 above.
                Example ouput:
                (#Recipe 1 Name#): (***  name ****)
                (#Description#):  (***  description ****)
                (#Recipe 2 Name#): (***  name ****)
                (#Description#):  (***  description ****)
                make sure to follow the (# #) and ( *** ***) in the output

                '''

            
           
            
            ),
       
    ]
)

In [52]:
chain = chat_prompt | model

In [53]:
response_from_model = chain.invoke(
    {
        
        "product": product,
        "food_info": food_info,
        "restrictions":restrictions
        
        
        

    }
)

In [55]:
recipe_2 = response_from_model.content


In [58]:
print(recipe_2)

Here are two vegan recipes using Parle-G biscuits:

(#Recipe 1 Name#): (**Parle-G Crumble Delight**)
(#Description#): (**A simple and delicious vegan crumble made with Parle-G biscuits, fruit, and a sweet and nutty crumble topping.**)

(#Recipe 2 Name#): (**Parle-G Vegan "Cheesecake"**)
(#Description#): (**A no-bake, vegan cheesecake made with Parle-G biscuits, cashews, and a creamy coconut milk filling.**) 



In [56]:
import re

In [60]:
pattern = r'\(#([^#]+)#\): \(\*\*([^*]+)\*\*\)'

# Find all matches
matches = re.findall(pattern, recipe_2)

# Convert matches into a dictionary of key-value pairs
result = {}
for key, value in matches:
    key = key.strip()
    value = value.strip()
    if key in result:
        result[key] += f" | {value}"  # Append if key already exists
    else:
        result[key] = value

# Output the result
for key, value in result.items():
    print(f'{key}: {value}')

Recipe 1 Name: Parle-G Crumble Delight
Description: A simple and delicious vegan crumble made with Parle-G biscuits, fruit, and a sweet and nutty crumble topping. | A no-bake, vegan cheesecake made with Parle-G biscuits, cashews, and a creamy coconut milk filling.
Recipe 2 Name: Parle-G Vegan "Cheesecake"


In [61]:
pattern = r'\(#([^#]+)#\): \(\*\*([^*]+)\*\*\)'

# Find all matches
matches = re.findall(pattern, recipe_2)

# Output the result in alternating format
for i in range(0, len(matches), 2):
    # Print recipe name
    recipe_name_key, recipe_name_value = matches[i]
    print(f'{recipe_name_key}: {recipe_name_value}')
    
    # Print description
    if i + 1 < len(matches):
        description_key, description_value = matches[i+1]
        print(f'{description_key}: {description_value}\n')

Recipe 1 Name: Parle-G Crumble Delight
Description: A simple and delicious vegan crumble made with Parle-G biscuits, fruit, and a sweet and nutty crumble topping.

Recipe 2 Name: Parle-G Vegan "Cheesecake"
Description: A no-bake, vegan cheesecake made with Parle-G biscuits, cashews, and a creamy coconut milk filling.

